In [ ]:
%matplotlib ipympl
import matplotlib as mpl
import matplotlib.pyplot as plt
import embag_python
import common.liegroups.se2_python as se2
from scipy.spatial.transform import Rotation as R
import experimental.beacon_sim.ekf_slam_python as esp
import experimental.beacon_sim.beacon_potential_python as bpp
import planning.probabilistic_road_map_python as prmp
import experimental.beacon_sim.correlated_beacons_python as cbp
import numpy as np

In [ ]:
bag_file = '/home/erick/code/robot-spot/2024-03-14-12-15-00.bag'


In [ ]:
def frame_from_child_at_time_from_ros_msg(ros_msg):
    # Convert the time to a float
    timestamp = ros_msg['header']['stamp'].to_sec()

    rot_vec = ros_msg["transform"]["rotation"]
    trans_vec= ros_msg["transform"]["translation"]
    parent_from_child_rot = R.from_quat([rot_vec['x'], rot_vec['y'], rot_vec['z'], rot_vec['w']])
    parent_from_child_trans = np.array([trans_vec['x'], trans_vec['y'], trans_vec['z']])

    x_axis_in_parent = parent_from_child_rot.apply(np.array([1.0, 0.0, 0.0]))
    theta = np.arctan2(x_axis_in_parent[1], x_axis_in_parent[0])

    parent_from_child = se2.SE2(theta, parent_from_child_trans[:2])
    return timestamp, parent_from_child 


In [ ]:
def estimate_from_msg(ros_msg):
    data = bytes(ros_msg.data()["estimate_proto"])
    return esp.EkfSlamEstimate.from_proto_string(data)

In [ ]:
bag = embag_python.View(bag_file)
print(bag)

last_map_msg = None
map_from_robot = []

for msg in bag.getMessages(['/tf', '/map']):
    if msg.topic == '/map':
        last_map_msg = msg

    if msg.topic == "/tf":
        for transform in msg.data()["transforms"]:
            if transform["header"]["frame_id"] not in ['map', 'flat_body']:
                continue
            if transform["child_frame_id"] not in ['map', 'flat_body']:
                continue
            time, parent_from_child = frame_from_child_at_time_from_ros_msg(transform)
            if transform["header"]["frame_id"] == "flat_body":
                map_from_robot_at_time = (time, parent_from_child.inverse())
            else:
                map_from_robot_at_time = (time, parent_from_child)

            map_from_robot.append(map_from_robot_at_time)


In [ ]:
estimate = estimate_from_msg(last_map_msg)

In [ ]:
def plot_map(estimate):
    ids = estimate.beacon_ids
    xs = []
    ys = []
    for i in ids:
        beacon_in_map = estimate.beacon_in_local(i)
        xs.append(beacon_in_map[0])
        ys.append(beacon_in_map[1])
        plt.text(beacon_in_map[0], beacon_in_map[1], f'tag_{i}')
    plt.plot(xs, ys, 'o')

def plot_trajectory(map_from_robot_at_time):
    for _, map_from_robot in map_from_robot_at_time[::5]:
        arrow_start = (map_from_robot @ np.array([0, 0])).flatten()
        x_axis = (map_from_robot @ np.array([1, 0])).flatten()
        delta = x_axis - arrow_start
        plt.arrow(arrow_start[0], arrow_start[1], delta[0], delta[1], head_width=0.25)

def plot_road_map(road_map):
    pts = np.stack(road_map.points())
    plt.plot(pts[:, 0], pts[:, 1], '*')

    line_segments = []
    for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if road_map.adj()[i, j] != 0:
                line_segments.append([pts[i, :], pts[j,:]])
    edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)
    ...

In [ ]:
road_map_pts = [
    np.array([[1.75], [0]]),
    np.array([[1.75], [-2]]),
    np.array([[1.75], [-5]]),
    np.array([[4.75], [-2.5]]),
    np.array([[4.75], [-4.75]])
]

adj = [
    (0, 1), (1, 3), (1,2), (1,4), (3, 4), (3, 2), (2, 4)
]

adj_mat = np.zeros((5, 5))

for pair in adj:
    adj_mat[pair[0], pair[1]] = 1.0
    adj_mat[pair[1], pair[0]] = 1.0
    
road_map = prmp.RoadMap(road_map_pts, adj_mat, None)

In [ ]:
exclusive_landmarks = cbp.BeaconClique(0.5, 1e-9, members = [1, 4])
absent_landmarks = cbp.BeaconClique(1e-9, 1 - 1e-9, [0, 2, 3, 5, 6, 7])

exclusive_potential = cbp.create_correlated_beacons(exclusive_landmarks)
absent_potential = cbp.create_correlated_beacons(absent_landmarks)

beacon_potential =  exclusive_potential * absent_potential 

In [ ]:
plt.figure()
plot_map(estimate)
# plot_trajectory(map_from_robot)
plot_road_map(road_map)

plt.plot(*zip(*road_map_pts), '*')
for i, pt in enumerate(road_map_pts):
    plt.text(*pt, f'{i}')
        
plt.axis('equal')

In [ ]:
with open('/tmp/robot_room_road_map.pb', 'wb') as file_out:
    file_out.write(road_map.to_proto_string())

with open('/tmp/robot_room_beacon_potential.pb', 'wb') as file_out:
    file_out.write(bea